In [1]:
import sys 
import os


sys.path.append(os.path.abspath(".."))

In [2]:
from src.data.load_data import load_raw_data

df=load_raw_data()

df.head()

,jira_created_at,work_start_at,pr_created_at,merged_at,pr_id,author_id,author_seniority,service_name,jira_issue_key,jira_type,...,is_legacy_codebase,coding_time_hours,review_time_hours,rework_time_hours,cycle_time_total,change_failure,trace_id,error_log,root_cause_category,failed_pipeline_step
0,2024-01-01 00:05:00,2024-01-02 18:19:32,2024-01-02 19:13:07,2024-01-02 19:46:10,PR_10000,DEV_040,Junior,Email-Dispatcher,APM-1000,Story,...,0,0.89,0.35,0.0,1.44,0,NaN,NaN,NaN,NaN
1,2024-01-01 00:15:00,2024-01-02 06:05:14,2024-01-02 07:15:18,2024-01-02 11:12:37,PR_10001,DEV_000,Mid,Notification-Broker,APM-1001,Tech Debt,...,1,1.17,2.00,0.0,5.12,0,NaN,NaN,NaN,NaN
2,2024-01-01 00:37:00,2024-01-02 02:35:50,2024-01-02 06:32:15,2024-01-02 15:00:10,PR_10002,DEV_044,Junior,Auth-Service,APM-1002,Tech Debt,...,1,3.94,4.96,0.0,12.41,1,TRC-ce59ec948b70,AssertionError: Auth-Service received unexpect...,AI_Hallucination_Logic,Unit_Test_Execution
3,2024-01-01 00:56:00,2024-01-02 06:17:52,2024-01-02 09:45:30,2024-01-03 01:20:44,PR_10003,DEV_020,Mid,Recommendation-Engine,APM-1003,Task,...,0,3.46,3.35,0.0,19.05,1,TRC-66fd4ac1eb13,AssertionError: Recommendation-Engine received...,AI_Hallucination_Logic,Unit_Test_Execution
4,2024-01-01 01:04:00,2024-01-02 07:23:44,2024-01-02 10:27:49,2024-01-02 19:33:32,PR_10004,DEV_033,Senior,Search-Optimizer,APM-1004,Story,...,0,3.07,2.15,0.0,12.16,0,NaN,NaN,NaN,NaN


In [3]:
from src.data.validate_data import validate_dataset

validate_dataset(df)

True

In [4]:
from src.data.split_data import split_dataset

X_train, X_test, y_train, y_test = split_dataset(df)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16000, 34)
(4000, 34)
(16000,)
(4000,)


In [5]:
from src.features.build_features import engineer_features, build_feature_pipeline

# Apply feature engineering first
X_train = engineer_features(X_train)
X_test = engineer_features(X_test)

# Build preprocessing pipeline
preprocessor = build_feature_pipeline(X_train)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(X_train_processed.shape)
print(X_test_processed.shape)

(16000, 46)
(4000, 46)


In [6]:
preprocessor = build_feature_pipeline(X_train)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(X_train_processed.shape)

(16000, 46)


In [7]:
df.groupby("jira_priority")["change_failure"].mean()

jira_priority
P0    0.060492
P1    0.053604
P2    0.065642
P3    0.055567
Name: change_failure, dtype: float64

In [8]:
from src.models.train import train_logistic_regression
from src.models.evaluate import evaluate_model

model = train_logistic_regression(X_train_processed, y_train)

evaluate_model(model, X_test_processed, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.69      0.81      3765
           1       0.11      0.62      0.19       235

    accuracy                           0.69      4000
   macro avg       0.54      0.66      0.50      4000
weighted avg       0.92      0.69      0.77      4000

ROC-AUC Score: 0.7177


0.7176818965273657

In [9]:
print("change_failure" in X_train.columns)
print("change_failure" in X_test.columns)

False
False


In [10]:
import pandas as pd

train_hash = pd.util.hash_pandas_object(X_train).sum()
test_hash = pd.util.hash_pandas_object(X_test).sum()

print(train_hash == test_hash)

False


In [11]:
len(set(X_train.index).intersection(set(X_test.index)))

0

In [12]:
df.groupby("change_failure")["root_cause_category"].count()

change_failure
0       0
1    1177
Name: root_cause_category, dtype: int64

In [13]:
df.groupby("change_failure")["failed_pipeline_step"].count()

change_failure
0       0
1    1177
Name: failed_pipeline_step, dtype: int64

In [14]:
df.groupby("change_failure")["failed_pipeline_step"].count()
df.groupby("change_failure")["harness_pipeline_status"].count()

change_failure
0    18823
1     1177
Name: harness_pipeline_status, dtype: int64

In [15]:
feature_names = preprocessor.get_feature_names_out()

In [16]:
import pandas as pd

coefficients = model.coef_[0]

feature_importance = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coefficients
})

feature_importance = feature_importance.sort_values(by="coefficient", ascending=False)

feature_importance.head(10)

,feature,coefficient
4,num__pr_size_loc,0.303472
16,num__pr_duration_hours,0.226346
19,cat__author_seniority_Junior,0.224369
14,num__rework_time_hours,0.211963
17,num__work_duration_hours,0.208967
9,num__task_complexity,0.192756
12,num__coding_time_hours,0.169875
13,num__review_time_hours,0.164160
11,num__is_legacy_codebase,0.156010
23,cat__service_name_Auth-Service,0.149093


In [17]:
from src.models.train import train_random_forest
from src.models.evaluate import evaluate_model

rf_model = train_random_forest(X_train_processed, y_train)

evaluate_model(rf_model, X_test_processed, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3765
           1       1.00      0.04      0.08       235

    accuracy                           0.94      4000
   macro avg       0.97      0.52      0.53      4000
weighted avg       0.95      0.94      0.92      4000

ROC-AUC Score: 0.7285


0.7285434149925124

In [18]:
import numpy as np
from sklearn.metrics import classification_report

y_proba = rf_model.predict_proba(X_test_processed)[:, 1]

# Try threshold = 0.2
y_pred_custom = (y_proba >= 0.2).astype(int)

print(classification_report(y_test, y_pred_custom))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      3765
           1       0.43      0.23      0.30       235

    accuracy                           0.94      4000
   macro avg       0.69      0.60      0.63      4000
weighted avg       0.92      0.94      0.93      4000



In [19]:
import numpy as np
from sklearn.metrics import classification_report

y_proba = rf_model.predict_proba(X_test_processed)[:, 1]

# Try threshold = 0.4
y_pred_custom = (y_proba >= 0.4).astype(int)

print(classification_report(y_test, y_pred_custom))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3765
           1       0.96      0.10      0.18       235

    accuracy                           0.95      4000
   macro avg       0.95      0.55      0.58      4000
weighted avg       0.95      0.95      0.93      4000



In [20]:
import numpy as np
from sklearn.metrics import classification_report

y_proba = rf_model.predict_proba(X_test_processed)[:, 1]

# Try threshold = 0.6
y_pred_custom = (y_proba >= 0.6).astype(int)

print(classification_report(y_test, y_pred_custom))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3765
           1       1.00      0.02      0.03       235

    accuracy                           0.94      4000
   macro avg       0.97      0.51      0.50      4000
weighted avg       0.95      0.94      0.92      4000



In [21]:
import xgboost

In [22]:
from src.models.train import train_xgboost
from src.models.evaluate import evaluate_model

xgb_model = train_xgboost(X_train_processed, y_train)

evaluate_model(xgb_model, X_test_processed, y_test)

C:\Users\jagdi\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [08:34:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.87      0.91      3765
           1       0.17      0.43      0.25       235

    accuracy                           0.85      4000
   macro avg       0.57      0.65      0.58      4000
weighted avg       0.91      0.85      0.88      4000

ROC-AUC Score: 0.7630


0.7630109349834704

In [23]:
import numpy as np
from sklearn.metrics import classification_report

y_proba = xgb_model.predict_proba(X_test_processed)[:, 1]

# Try threshold 0.3
y_pred_custom = (y_proba >= 0.3).astype(int)

print(classification_report(y_test, y_pred_custom))

              precision    recall  f1-score   support

           0       0.97      0.68      0.80      3765
           1       0.12      0.68      0.20       235

    accuracy                           0.68      4000
   macro avg       0.54      0.68      0.50      4000
weighted avg       0.92      0.68      0.77      4000



In [24]:
from src.risk.risk_scoring import generate_risk_output

# Get probability from XGBoost
sample_proba = xgb_model.predict_proba(X_test_processed[:1])[:, 1][0]

generate_risk_output(sample_proba)

{'failure_probability': 0.751,
 'risk_score': 75.1,
 'risk_category': 'High Risk'}

In [25]:
from src.models.predict import predict_risk

# Take one raw sample from test set
sample_input = df.iloc[[0]]   # raw, NOT processed

predict_risk(xgb_model, preprocessor, sample_input)

{'failure_probability': 0.0033,
 'risk_score': 0.33,
 'risk_category': 'Low Risk'}

In [26]:
pip install fastapi uvicorn


   ------------------------ --------------- 3/5 [starlette]
   -------------------------------- ------- 4/5 [fastapi]
   ---------------------------------------- 5/5 [fastapi]

Note: you may need to restart the kernel to use updated packages.


In [27]:
import fastapi

In [30]:
import joblib

# Save model
joblib.dump(xgb_model, "../models/xgb_model.pkl")

# Save preprocessor
joblib.dump(preprocessor, "../models/preprocessor.pkl")

['../models/preprocessor.pkl']

In [29]:
import os
print(os.getcwd())

C:\Users\jagdi\ai-assisted-code-risk-prediction\notebooks


In [31]:
selected_features = [
    "task_complexity",
    "pr_size_loc",
    "rework_time_hours",
    "coding_time_hours",
    "review_time_hours",
    "deployment_pressure",
    "ai_acceptance_rate",
    "num_dependencies",
    "author_seniority",
    "service_name",
    "is_legacy_codebase"
]

X = df[selected_features]
y = df["change_failure"]

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numeric_cols = [
    "task_complexity",
    "pr_size_loc",
    "rework_time_hours",
    "coding_time_hours",
    "review_time_hours",
    "deployment_pressure",
    "ai_acceptance_rate",
    "num_dependencies"
]

categorical_cols = [
    "author_seniority",
    "service_name",
    "is_legacy_codebase"
]

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_cols),
    ("cat", categorical_pipeline, categorical_cols)
])

In [34]:
from xgboost import XGBClassifier

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

model = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight= (y_train.value_counts()[0] / y_train.value_counts()[1])
)

model.fit(X_train_processed, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [35]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test_processed)
y_proba = model.predict_proba(X_test_processed)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.96      0.82      0.88      3765
           1       0.14      0.49      0.22       235

    accuracy                           0.80      4000
   macro avg       0.55      0.65      0.55      4000
weighted avg       0.91      0.80      0.85      4000

ROC-AUC: 0.753029866350202


In [36]:
import joblib

joblib.dump(model, "../models/xgb_model.pkl")
joblib.dump(preprocessor, "../models/preprocessor.pkl")

['../models/preprocessor.pkl']

In [37]:
y_proba = model.predict_proba(X_test_processed)[:, 1]

In [38]:
import numpy as np
from sklearn.metrics import classification_report

thresholds = np.arange(0.1, 0.9, 0.05)

for t in thresholds:
    y_pred_custom = (y_proba >= t).astype(int)
    
    print(f"\nThreshold: {t:.2f}")
    print(classification_report(y_test, y_pred_custom))


Threshold: 0.10
              precision    recall  f1-score   support

           0       0.98      0.34      0.51      3765
           1       0.08      0.91      0.15       235

    accuracy                           0.38      4000
   macro avg       0.53      0.63      0.33      4000
weighted avg       0.93      0.38      0.49      4000


Threshold: 0.15
              precision    recall  f1-score   support

           0       0.98      0.42      0.59      3765
           1       0.09      0.87      0.16       235

    accuracy                           0.45      4000
   macro avg       0.53      0.65      0.37      4000
weighted avg       0.93      0.45      0.57      4000


Threshold: 0.20
              precision    recall  f1-score   support

           0       0.98      0.48      0.64      3765
           1       0.09      0.83      0.16       235

    accuracy                           0.50      4000
   macro avg       0.53      0.65      0.40      4000
weighted avg       0.93

In [39]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

In [44]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []

for train_idx, val_idx in skf.split(X, y):
    X_train_fold = X.iloc[train_idx]
    X_val_fold = X.iloc[val_idx]
    y_train_fold = y.iloc[train_idx]
    y_val_fold = y.iloc[val_idx]

    # Feature engineering
    X_train_fold = engineer_features(X_train_fold)
    X_val_fold = engineer_features(X_val_fold)

    # Fit preprocessor on training fold only
    preprocessor = build_feature_pipeline(X_train_fold)
    X_train_processed = preprocessor.fit_transform(X_train_fold)
    X_val_processed = preprocessor.transform(X_val_fold)


    negative = (y == 0).sum()
    positive = (y == 1).sum()

    scale_pos_weight = negative / positive

    print("scale_pos_weight:", scale_pos_weight)

    # Train model
    model = XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        eval_metric="logloss"
    )
    model.fit(X_train_processed, y_train_fold)

    # Evaluate
    y_val_proba = model.predict_proba(X_val_processed)[:, 1]
    auc = roc_auc_score(y_val_fold, y_val_proba)

    auc_scores.append(auc)

print("AUC Scores per fold:", auc_scores)
print("Mean AUC:", np.mean(auc_scores))
print("Std AUC:", np.std(auc_scores))

scale_pos_weight: 15.992353440951572
scale_pos_weight: 15.992353440951572
scale_pos_weight: 15.992353440951572
scale_pos_weight: 15.992353440951572
scale_pos_weight: 15.992353440951572
AUC Scores per fold: [0.7389104574609364, 0.7183843350004239, 0.7448390833827807, 0.720006889533313, 0.7246213008159368]
Mean AUC: 0.7293524132386782
Std AUC: 0.010594471575604971


In [45]:
param_dist = {
    "max_depth": [3, 4, 5],
    "learning_rate": [0.05, 0.1],
    "n_estimators": [150, 200],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

In [46]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric="logloss"
)

In [47]:
search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,              # small & safe
    scoring="roc_auc",
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

search.fit(X_train_processed, y_train)

print("Best Parameters:", search.best_params_)
print("Best CV AUC:", search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'subsample': 1.0, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
Best CV AUC: 0.5064164457431223


In [48]:
best_model = search.best_estimator_

In [49]:
from sklearn.metrics import roc_auc_score

y_test_proba = best_model.predict_proba(X_test_processed)[:, 1]
test_auc = roc_auc_score(y_test, y_test_proba)

print("Test ROC-AUC:", test_auc)

ValueError: Feature shape mismatch, expected: 24, got 25

In [50]:
from sklearn.pipeline import Pipeline

In [51]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        eval_metric="logloss"
    ))
])

In [52]:
param_dist = {
    "model__max_depth": [3, 4, 5],
    "model__learning_rate": [0.05, 0.1],
    "model__n_estimators": [150, 200],
    "model__subsample": [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0]
}

In [53]:
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    scoring="roc_auc",
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'model__colsample_bytree': [0.8, 1.0], 'model__learning_rate': [0.05, 0.1], 'model__max_depth': [3, 4, ...], 'model__n_estimators': [150, 200], ...}"
,n_iter,10
,scoring,'roc_auc'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [54]:
best_model = search.best_estimator_

y_test_proba = best_model.predict_proba(X_test)[:, 1]

from sklearn.metrics import roc_auc_score
print("Test ROC-AUC:", roc_auc_score(y_test, y_test_proba))

Test ROC-AUC: 0.7754779463705462


## After validating baseline performance using 5-fold stratified cross-validation (mean AUC ~0.73), I performed controlled hyperparameter tuning using RandomizedSearchCV. This improved the final test ROC-AUC to ~0.775.

In [59]:
y_proba = best_model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.1, 0.9, 0.05)

for t in thresholds:
    y_pred_custom = (y_proba >= t).astype(int)
    print(f"\nThreshold: {t:.2f}")
    print(classification_report(y_test, y_pred_custom))


Threshold: 0.10
              precision    recall  f1-score   support

           0       1.00      0.15      0.26      3765
           1       0.07      0.99      0.13       235

    accuracy                           0.20      4000
   macro avg       0.53      0.57      0.19      4000
weighted avg       0.94      0.20      0.25      4000


Threshold: 0.15
              precision    recall  f1-score   support

           0       1.00      0.21      0.34      3765
           1       0.07      0.99      0.13       235

    accuracy                           0.25      4000
   macro avg       0.53      0.60      0.24      4000
weighted avg       0.94      0.25      0.33      4000


Threshold: 0.20
              precision    recall  f1-score   support

           0       0.99      0.27      0.42      3765
           1       0.08      0.96      0.14       235

    accuracy                           0.31      4000
   macro avg       0.53      0.62      0.28      4000
weighted avg       0.94

In [66]:
from sklearn.metrics import classification_report

thresholds = [0.25, 0.30, 0.35, 0.40, 0.45]

for t in thresholds:
    y_pred = (y_proba >= t).astype(int)
    
    print(f"\nThreshold: {t}")
    print(classification_report(y_test, y_pred))


Threshold: 0.25
              precision    recall  f1-score   support

           0       0.99      0.34      0.50      3765
           1       0.08      0.92      0.15       235

    accuracy                           0.37      4000
   macro avg       0.53      0.63      0.32      4000
weighted avg       0.93      0.37      0.48      4000


Threshold: 0.3
              precision    recall  f1-score   support

           0       0.98      0.40      0.57      3765
           1       0.08      0.89      0.15       235

    accuracy                           0.43      4000
   macro avg       0.53      0.64      0.36      4000
weighted avg       0.93      0.43      0.54      4000


Threshold: 0.35
              precision    recall  f1-score   support

           0       0.98      0.46      0.63      3765
           1       0.09      0.84      0.16       235

    accuracy                           0.49      4000
   macro avg       0.53      0.65      0.39      4000
weighted avg       0.93 

### After hyperparameter tuning improved ROC-AUC to ~0.775, I re-optimized the probability threshold. At 0.45, recall remained high (~0.76) while reducing false positives compared to lower thresholds. This provided a better business-aligned trade-off.